In [1]:
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
import torch
import torch.nn as nn
import torch.optim as optim
import cv2
from IPython.display import HTML
import matplotlib.animation as animation
from tqdm.notebook import tqdm
from torchvision.utils import save_image
from torchvision.utils import make_grid
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(style='darkgrid', font_scale=1.2)

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
latent_size = 64

In [4]:
def get_dataloader(image_size, batch_size):

    root = 'D:/faces/'
    stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
    dataset = ImageFolder(root, transform=tt.Compose([
        tt.Resize(image_size),
        tt.RandomHorizontalFlip(p=0.5),
        tt.ToTensor(),
        tt.Normalize(*stats)
    ]))

    dataloader = DataLoader(dataset, batch_size, num_workers=4, pin_memory=True)

#   """
#   Builds dataloader for training data.
#   Use tt.Compose and tt.Resize for transformations
#   :param image_size: height and wdith of the image
#   :param batch_size: batch_size of the dataloader
#   :returns: DataLoader object 
#   """
    return dataloader
  # TODO: resize images, convert them to tensors and build dataloader

In [5]:
image_size = 64
batch_size = 64

In [6]:
class Generator(nn.Module):
    def __init__(self,):
        super(Generator, self).__init__()
#         self.ngpu = ngpu
        self.main = nn.Sequential(
        # in latent_size  x 1 x 1

        # out = (in - 1) x stride(1) - 2 x padding(0) + dilation(1) x (kernel(6) - 1) + output_padding + 1

        nn.ConvTranspose2d(latent_size, 256, kernel_size=6,
                           stride=1, padding=0, bias=False), # (1-1)x1 - 2x0 + 1x5 + 0 +1 = 6
        # out 256 x 6 x 6
        nn.BatchNorm2d(256),
        nn.ReLU(True),
        nn.UpsamplingBilinear2d(scale_factor=4), 
        # out 6 x 4 = 24
        nn.Conv2d(256, 256 , kernel_size=2, stride=3 , padding=1),  # (24 - 2 + 2)/ 3 +1 = 9
        nn.BatchNorm2d(256),
        nn.ReLU(True),
        # out 256 x 9 x 9

        nn.UpsamplingBilinear2d(scale_factor=4),
        # 9 x 4 = 36
        nn.Conv2d(256, 128, kernel_size=6, stride=2, padding=0, bias=False), # (36 - 6 + 0)/ 2 +1 = 16
        nn.BatchNorm2d(128),
        nn.ReLU(True),
        # 128 x 16 x 16

        nn.UpsamplingBilinear2d(scale_factor=4),
        # 16 x 4 = 64
        nn.Conv2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False), # (64 - 4 + 2)/ 2 +1 = 32
        nn.BatchNorm2d(64),
        nn.ReLU(True),
        # 64 x 32 x 32

        nn.UpsamplingBilinear2d(scale_factor=4),
        # 32 x 4 = 128
        nn.Conv2d(64, 64, kernel_size=4, stride=2, padding=1, bias=False), # (128 - 4 + 2)/ 2 +1 = 64
        nn.BatchNorm2d(64),
        nn.ReLU(True),
        # 64 x 64 x 64

        nn.UpsamplingBilinear2d(scale_factor=2),
        # 64 x 2 = 128
        nn.Conv2d(64, 32, kernel_size=4, stride=2, padding=1, bias=False), # (128 - 4 + 2)/ 2 +1 = 64
        nn.BatchNorm2d(32),
        nn.ReLU(True),
        # 32 x 64 x 64

#         nn.UpsamplingBilinear2d(scale_factor=4),
#         # 64 x 2 = 128
#         nn.Conv2d(32, 16, kernel_size=2, stride=2, padding=1, bias=False), # (128 - 4 + 2)/ 2 +1 = 64
#         nn.BatchNorm2d(16),
#         nn.ReLU(True),
#         # 16 x 64 x 64

        nn.ConvTranspose2d(32, 3, kernel_size=1, stride=1, padding=0, bias=False), # (63x1 - 0 + 1x(1-1) + 0 +1 = 64
        nn.Tanh()
        # out: 3 x 64 x 64


    )

    def forward(self, input):
        return self.main(input)

In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
        # in 3 x 128 x 128

        nn.Conv2d(3, 256 , kernel_size=2, stride=2, padding=2, bias=False), # (64 - 2 + 4)/3  +1 = 34
        nn.BatchNorm2d(256),
        nn.ReLU(),  

        # out 256 x 34 x 34

        nn.Conv2d(256, 512, kernel_size=2, stride=3, padding=2, bias=False), # (34 - 2 + 4 )/ 3 +1 = 13
        nn.BatchNorm2d(512),
        nn.ReLU(), 

        #out 512 x 13 x 13

        nn.Conv2d(512, 512, kernel_size=3, stride=3, padding=1, bias=False), # (13 - 3 + 2 )/ 3 +1 = 5
        nn.BatchNorm2d(512),
        nn.ReLU(), 

        #out 512 x 5 x 5

        nn.Conv2d(512, 1 , kernel_size=5, stride=1 , padding=0, bias=False), # (5-5+0 )/1 +1 = 1

        # out 1 x 1 x 1
#         nn.Flatten(),
        nn.Sigmoid()
    )
    def forward(self, input):
        return self.main(input)

In [8]:
model_G = torch.load('./generator').to('cpu')
model_D = torch.load('./diskriminator').to('cpu')

In [9]:
n_images = 100

noise = torch.randn(n_images, latent_size, 1, 1, device='cpu')

In [10]:
# fig = plt.figure(figsize=(8,8))
# plt.axis("off")
# plt.imshow((model_G(noise)[-5].detach().cpu().numpy().reshape(64,64,3)))
# plt.show();

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
X_fake = model_G(noise)
y_fake = np.zeros(n_images)

In [12]:
fake_list = []
     
with torch.no_grad():
    print(n)
    fake = model_G(n).detach().cpu()
fake_list.append(vutils.make_grid(fake, padding=2, normalize=True))
    

NameError: name 'n' is not defined

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(model_G(noise)[7].detach().cpu().numpy().reshape(64,64,3))
plt.show()